In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)

In [2]:
!pip install -q gdown

import gdown

# ID file trên Google Drive
file_id = "1yPPJvcMIxFefGK6V75Mn0cqq7FbpgKEu"

# Tạo URL dạng direct download
url = f"https://drive.google.com/uc?id={file_id}"

# Tên file bạn muốn lưu về
output = "data_auto_mpg.csv"   # đổi tên nếu bạn muốn

gdown.download(url, output, quiet=False)



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\hangu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Downloading...
From: https://drive.google.com/uc?id=1yPPJvcMIxFefGK6V75Mn0cqq7FbpgKEu
To: c:\Users\hangu\OneDrive\Documents\0. Personal documents\0.6 ML AI\2. AIO2025\2.2 Weekly exercise\12. Auto_MPG\data_auto_mpg.csv
100%|██████████| 15.4k/15.4k [00:00<00:00, 1.05MB/s]


'data_auto_mpg.csv'

In [3]:
dataset = pd.read_csv(output)
display(dataset)

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
0,18.0,8,307.0,130.0,3504.0,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,0,0,1
...,...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86.0,2790.0,15.6,82,0,0,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,1,0,0
389,32.0,4,135.0,84.0,2295.0,11.6,82,0,0,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,0,0,1


In [4]:
X = dataset.drop("MPG", axis=1).values
y = dataset["MPG"].values

In [5]:
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=val_size, random_state=random_state, shuffle=is_shuffle)
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=test_size, random_state=random_state, shuffle=is_shuffle)

In [6]:
normalizer = StandardScaler()
X_train_scaled = normalizer.fit_transform(X_train)
X_val_scaled = normalizer.transform(X_val)
X_test_scaled = normalizer.transform(X_test)   

In [7]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
        self.n_samples = X.shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [8]:
BATCH_SIZE = 32
train_dataset = CustomDataset(X_train_scaled, y_train)
val_dataset   = CustomDataset(X_val_scaled,   y_val)
test_dataset  = CustomDataset(X_test_scaled,  y_test)


In [9]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        
        if isinstance(hidden_dim, int):
            h1 = h2 = hidden_dim
        else:
            h1, h2 = hidden_dim

        self.fc1 = nn.Linear(input_dim, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, output_dim)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = x.squeeze(1)
        return x

In [13]:
input_dim = X_train_scaled.shape[1]
model = MLP(input_dim=input_dim, hidden_dim=64, output_dim=1).to(device)
print(model)


MLP(
  (fc1): Linear(in_features=9, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)


In [14]:
def r_squared(y_true, y_pred):
    y_true = y_true.view(-1).float()
    y_pred = y_pred.view(-1).float()

    ss_res = torch.sum((y_true - y_pred) ** 2)
    ss_tot = torch.sum((y_true - torch.mean(y_true)) ** 2)
    if ss_tot == 0:
        return 0.0
    return (1 - ss_res / ss_tot).item()

In [15]:
NUM_EPOCHS = 100
LEARNING_RATE = 0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [16]:
for epoch in range(NUM_EPOCHS + 1):
    model.train()
    train_losses = []
    all_preds = []
    all_targets = []
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_batch = y_batch.squeeze(1)

        optimizer.zero_grad()
        outputs = model(X_batch) 
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        all_preds.append(outputs.detach().cpu())
        all_targets.append(y_batch.detach().cpu())

    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    train_r2 = r_squared(all_targets, all_preds)
    avg_train_loss = np.mean(train_losses)

    model.eval()
    val_losses = []
    all_val_preds = []
    all_val_targets = []
    
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch = X_val_batch.to(device)
            y_val_batch = y_val_batch.to(device)
            y_val_batch = y_val_batch.squeeze(1)

            val_outputs = model(X_val_batch)
            val_loss = criterion(val_outputs, y_val_batch)

            val_losses.append(val_loss.item())
            all_val_preds.append(val_outputs.detach().cpu())
            all_val_targets.append(y_val_batch.detach().cpu())
        
        all_val_preds = torch.cat(all_val_preds)
        all_val_targets = torch.cat(all_val_targets)
        val_r2 = r_squared(all_val_targets, all_val_preds)
        avg_val_loss = np.mean(val_losses)

    print(f"Epoch [{epoch}/{NUM_EPOCHS}] "
          f"Train Loss: {avg_train_loss:.4f}, Train R2: {train_r2:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Val R2: {val_r2:.4f}")
    

Epoch [0/100] Train Loss: 610.7095, Train R2: -9.3316 | Val Loss: 616.7402, Val R2: -9.0984
Epoch [1/100] Train Loss: 594.9578, Train R2: -9.0581 | Val Loss: 598.2004, Val R2: -8.7985
Epoch [2/100] Train Loss: 575.6774, Train R2: -8.6870 | Val Loss: 570.8436, Val R2: -8.3569
Epoch [3/100] Train Loss: 544.0577, Train R2: -8.1528 | Val Loss: 531.0790, Val R2: -7.7162
Epoch [4/100] Train Loss: 495.5658, Train R2: -7.3856 | Val Loss: 476.5319, Val R2: -6.8368
Epoch [5/100] Train Loss: 436.7393, Train R2: -6.3608 | Val Loss: 406.5802, Val R2: -5.7066
Epoch [6/100] Train Loss: 361.0905, Train R2: -5.1152 | Val Loss: 322.2185, Val R2: -4.3377
Epoch [7/100] Train Loss: 270.5188, Train R2: -3.6587 | Val Loss: 232.4997, Val R2: -2.8700
Epoch [8/100] Train Loss: 188.9214, Train R2: -2.1915 | Val Loss: 150.4982, Val R2: -1.5122
Epoch [9/100] Train Loss: 119.2448, Train R2: -0.9912 | Val Loss: 89.4822, Val R2: -0.4800
Epoch [10/100] Train Loss: 68.5533, Train R2: -0.1729 | Val Loss: 57.4763, Val R2

In [17]:
model.eval()
all_test_preds = []
all_test_targets = []

with torch.no_grad():
    for X_test_batch, y_test_batch in test_loader:
        X_test_batch = X_test_batch.to(device)
        y_test_batch = y_test_batch.to(device)
        y_test_batch = y_test_batch.squeeze(1)

        test_outputs = model(X_test_batch)

        all_test_preds.append(test_outputs.detach().cpu())
        all_test_targets.append(y_test_batch.detach().cpu())
    
all_test_preds = torch.cat(all_test_preds)
all_test_targets = torch.cat(all_test_targets)
test_r2 = r_squared(all_test_targets, all_test_preds)
print(f"Test R2: {test_r2:.4f}")

Test R2: 0.8837
